In [1]:
import numpy as np
import pandas as pd


Insight: Its really useful (important?) to have a good idea of the solution space

In [2]:
x1 = np.random.randint(20,size=(50))
x2 = np.random.randint(80,size=(50))
x3 = np.random.randint(10,size=(50))
x4 = np.random.randint(100,size=(50))
x5 = np.random.randint(130,size=(50))
x6 = np.random.randint(5,size=(50))


y1 = 2*x1 + 3*x2 + 4*x3 + np.random.normal(1,1) + 6*x4 + 3*x5 + 6*x6

y2 = 7*x1 - 2*x2 + x3

In [3]:
XX1 =  {'col1': x1, 'col2': x2, 'col3': x3, 'col4': x4, 'col5': x5, 'col6': x6}
XX1 = pd.DataFrame(data= XX1)
XX1_numpy = XX1.to_numpy()

YY1 = {'col1': y1 }
YY1 = pd.DataFrame(data= YY1)
YY1_numpy = YY1.to_numpy()

YY2 = {'col1': y2 }
YY2 = pd.DataFrame(data= YY2)

In [4]:
YY2

,col1
0,-37
1,-41
2,13
3,41
4,14
5,45
6,-1
7,-67
8,85
9,107


In [5]:
input = XX1

In [6]:
weights_size = len(input.iloc[0])
solutions_per_problem = 100

pop_size = (solutions_per_problem, weights_size)
new_population = np.random.randint(low = 1, high= 20, size = pop_size)

pop_original = new_population

In [7]:
new_population

array([[19, 10, 16, 12, 11, 19],
       [ 7,  7, 17, 10,  9,  9],
       [15,  4,  1,  5, 16, 12],
       [ 1, 11,  8,  6,  4, 14],
       [16, 16, 17, 15, 19,  2],
       [ 4,  4, 15, 10, 17, 10],
       [ 1, 13,  4, 10,  6,  2],
       [13, 18,  9, 16,  6, 19],
       [14, 12,  2,  7,  8,  1],
       [17,  2, 15, 16, 13, 18],
       [17,  9,  2, 16, 16, 15],
       [10, 16,  9,  3,  6,  6],
       [15,  4, 16,  1, 13,  2],
       [ 2, 17, 12, 10,  2, 13],
       [ 2, 10,  9,  1, 17, 18],
       [10,  4,  1,  2, 18,  1],
       [ 4,  1, 12,  1,  8,  7],
       [ 8,  2,  9,  3,  3,  3],
       [12,  3, 11, 12,  9,  8],
       [ 5, 19, 13, 12,  2,  3],
       [16,  5, 19,  5,  6, 19],
       [ 2,  7, 18, 14, 15, 11],
       [ 1,  4,  8,  3, 15, 15],
       [ 1, 12,  3,  5,  8,  5],
       [19,  7, 19,  3,  1,  1],
       [ 4, 18, 17, 12, 19, 15],
       [19, 14, 18, 15,  1, 16],
       [19,  6, 12,  3, 17,  3],
       [10,  4,  2,  9,  1, 14],
       [16, 14,  5, 16,  2,  5],
       [16

In [8]:
# fitness of each population:

fitness = np.empty(new_population.shape[0])

def cal_pop_fitness(target_dataframe, electricity_values, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calculates the sum of products between each input and its corresponding weight.
    generated_electricity = np.sum(pop*electricity_values, axis=1);
    required_electricity =  target_dataframe['col1']
    error = required_electricity - generated_electricity
    error = error.abs()
    error = error.sum()
    fitness = 1/error
    
    return fitness


for i in range(len(new_population)):
    fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])

fitness

array([1.63224125e-05, 2.56947520e-05, 2.40908980e-05, 5.73675640e-05,
       1.01649308e-05, 1.85272836e-05, 2.89994160e-05, 1.50230334e-05,
       2.70823240e-05, 1.62023471e-05, 1.26688997e-05, 3.47031152e-05,
       3.67198043e-05, 2.99056897e-05, 2.38854119e-05, 2.66821348e-05,
       1.02862648e-04, 1.75499019e-04, 2.61701642e-05, 2.33833604e-05,
       4.85355536e-05, 1.58069146e-05, 3.38903064e-05, 3.82455148e-05,
       1.10363323e-04, 1.11989039e-05, 2.07553050e-05, 2.14159567e-05,
       1.13825648e-04, 2.07505675e-05, 1.97366519e-05, 6.10408496e-05,
       1.49520518e-05, 4.60323417e-05, 1.70629674e-05, 1.19346763e-05,
       2.26498242e-05, 1.54127251e-05, 3.34923565e-05, 4.17915877e-05,
       2.73591398e-05, 1.08562553e-04, 2.14361410e-04, 2.21962976e-05,
       1.30660109e-05, 2.12321198e-05, 2.84734368e-05, 2.53482513e-05,
       2.06311255e-05, 2.19161495e-05, 1.27789888e-05, 2.33097771e-05,
       2.21960038e-05, 1.94668220e-05, 1.48448447e-05, 2.52203928e-05,
      

In [9]:
np.max(fitness)

0.0002377784127664372

In [10]:
new_population

array([[19, 10, 16, 12, 11, 19],
       [ 7,  7, 17, 10,  9,  9],
       [15,  4,  1,  5, 16, 12],
       [ 1, 11,  8,  6,  4, 14],
       [16, 16, 17, 15, 19,  2],
       [ 4,  4, 15, 10, 17, 10],
       [ 1, 13,  4, 10,  6,  2],
       [13, 18,  9, 16,  6, 19],
       [14, 12,  2,  7,  8,  1],
       [17,  2, 15, 16, 13, 18],
       [17,  9,  2, 16, 16, 15],
       [10, 16,  9,  3,  6,  6],
       [15,  4, 16,  1, 13,  2],
       [ 2, 17, 12, 10,  2, 13],
       [ 2, 10,  9,  1, 17, 18],
       [10,  4,  1,  2, 18,  1],
       [ 4,  1, 12,  1,  8,  7],
       [ 8,  2,  9,  3,  3,  3],
       [12,  3, 11, 12,  9,  8],
       [ 5, 19, 13, 12,  2,  3],
       [16,  5, 19,  5,  6, 19],
       [ 2,  7, 18, 14, 15, 11],
       [ 1,  4,  8,  3, 15, 15],
       [ 1, 12,  3,  5,  8,  5],
       [19,  7, 19,  3,  1,  1],
       [ 4, 18, 17, 12, 19, 15],
       [19, 14, 18, 15,  1, 16],
       [19,  6, 12,  3, 17,  3],
       [10,  4,  2,  9,  1, 14],
       [16, 14,  5, 16,  2,  5],
       [16

In [11]:
# Select the best parents for mating process:

def select_mating_pool(pop, fitness, number_of_parents):
    parents = np.empty((number_of_parents, new_population.shape[1]))

    for instance in range(number_of_parents):
        max_index = np.where(fitness == np.max(fitness))
        max_index = max_index[0][0]
        parents[instance, :] = pop[max_index, :]
        fitness[max_index] = -9999

    return parents

In [12]:
max_index = np.where(fitness == np.max(fitness))
max_index = max_index[0][0]
max_index

90

In [13]:
# Select the best parents for mating process:

def pseudo_select_mating_pool(pop, fitness, number_of_parents):
    parents = np.empty((number_of_parents, new_population.shape[1]))

    for instance in range(number_of_parents):
        max_index = np.where(fitness == np.max(fitness))
        max_index = max_index[0][0]
        parents[instance, :] = pop[instance, :]
        fitness[max_index] = -9999

    return fitness



In [14]:
# The mating process with each parent

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    
    # Choose mating (crossover) point randomly:
    
    division_point = np.random.randint(low = 2, high= (parents.shape[1]) - 1 )
    
    
    crossover_point = np.uint8(offspring_size[1]/ division_point )
 
    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [15]:
# Mutations on the offsprings:

mutation_probability = 100 # In percent

def mutation(offspring_crossover):
    
    offspring_operated = offspring_crossover

    # Mutation changes a single gene in each offspring randomly.
    
    prob = np.random.randint(low = 0, high = 100)
    
    if prob < mutation_probability :
        
        for idx in range(offspring_operated.shape[0]):

        # The random value to be added to the gene.

            random_value = np.random.randint(low = -10 , high= 10)
            
            # select the column to be mutated
            
            col = np.random.randint(low = 0, high = (offspring_crossover.shape[1] - 1))

            offspring_operated[idx, col] = offspring_operated[idx, col] + random_value
                  

    return offspring_operated

In [16]:
num_par = 4

test_mating_pool = select_mating_pool(new_population, fitness, num_par )

test_mating_pool

array([[ 3.,  6., 19.,  4.,  2.,  3.],
       [ 6.,  5., 16.,  4.,  1.,  1.],
       [ 6.,  1., 15.,  3.,  4., 11.],
       [ 8.,  2.,  9.,  3.,  3.,  3.]])

In [17]:
fitness

array([ 1.63224125e-05,  2.56947520e-05,  2.40908980e-05,  5.73675640e-05,
        1.01649308e-05,  1.85272836e-05,  2.89994160e-05,  1.50230334e-05,
        2.70823240e-05,  1.62023471e-05,  1.26688997e-05,  3.47031152e-05,
        3.67198043e-05,  2.99056897e-05,  2.38854119e-05,  2.66821348e-05,
        1.02862648e-04, -9.99900000e+03,  2.61701642e-05,  2.33833604e-05,
        4.85355536e-05,  1.58069146e-05,  3.38903064e-05,  3.82455148e-05,
        1.10363323e-04,  1.11989039e-05,  2.07553050e-05,  2.14159567e-05,
        1.13825648e-04,  2.07505675e-05,  1.97366519e-05,  6.10408496e-05,
        1.49520518e-05,  4.60323417e-05,  1.70629674e-05,  1.19346763e-05,
        2.26498242e-05,  1.54127251e-05,  3.34923565e-05,  4.17915877e-05,
        2.73591398e-05,  1.08562553e-04, -9.99900000e+03,  2.21962976e-05,
        1.30660109e-05,  2.12321198e-05,  2.84734368e-05,  2.53482513e-05,
        2.06311255e-05,  2.19161495e-05,  1.27789888e-05,  2.33097771e-05,
        2.21960038e-05,  

In [18]:
crossover_pop = crossover(test_mating_pool, offspring_size=(pop_size[0]- test_mating_pool.shape[0], weights_size))
crossover_pop 

array([[ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  4., 11.],
       [ 6.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3

In [19]:
mutation(crossover_pop)

array([[ 3.,  6., 19., -2.,  1.,  1.],
       [ 6.,  5., 16.,  3.,  6., 11.],
       [ 6.,  1., 15.,  9.,  3.,  3.],
       [ 8.,  2., 16.,  4.,  2.,  3.],
       [ 3.,  6., 19.,  3.,  1.,  1.],
       [ 6.,  5., 15.,  3.,  4., 11.],
       [ 6., -6., 15.,  3.,  3.,  3.],
       [ 1.,  2.,  9.,  4.,  2.,  3.],
       [ 3.,  2., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16., -1.,  4., 11.],
       [ 6.,  1., 15.,  7.,  3.,  3.],
       [-1.,  2.,  9.,  4.,  2.,  3.],
       [ 3., 14., 19.,  4.,  1.,  1.],
       [-4.,  5., 16.,  3.,  4., 11.],
       [ 6.,  7., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  4.,  4.,  2.,  3.],
       [ 9.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 17.,  3.,  4., 11.],
       [ 9.,  1., 15.,  3.,  3.,  3.],
       [ 8.,  2.,  9.,  4.,  5.,  3.],
       [ 3.,  6., 19., 10.,  1.,  1.],
       [ 6.,  5., 11.,  3.,  4., 11.],
       [ 6.,  1., 10.,  3.,  3.,  3.],
       [16.,  2.,  9.,  4.,  2.,  3.],
       [-6.,  6., 19.,  4.,  1.,  1.],
       [ 6.,  5., 16.,  3

In [20]:
new_population[0:test_mating_pool.shape[0], :] = test_mating_pool
new_population[test_mating_pool.shape[0]:, :] = crossover_pop 
new_population

array([[ 3,  6, 19,  4,  2,  3],
       [ 6,  5, 16,  4,  1,  1],
       [ 6,  1, 15,  3,  4, 11],
       [ 8,  2,  9,  3,  3,  3],
       [ 3,  6, 19, -2,  1,  1],
       [ 6,  5, 16,  3,  6, 11],
       [ 6,  1, 15,  9,  3,  3],
       [ 8,  2, 16,  4,  2,  3],
       [ 3,  6, 19,  3,  1,  1],
       [ 6,  5, 15,  3,  4, 11],
       [ 6, -6, 15,  3,  3,  3],
       [ 1,  2,  9,  4,  2,  3],
       [ 3,  2, 19,  4,  1,  1],
       [ 6,  5, 16, -1,  4, 11],
       [ 6,  1, 15,  7,  3,  3],
       [-1,  2,  9,  4,  2,  3],
       [ 3, 14, 19,  4,  1,  1],
       [-4,  5, 16,  3,  4, 11],
       [ 6,  7, 15,  3,  3,  3],
       [ 8,  2,  4,  4,  2,  3],
       [ 9,  6, 19,  4,  1,  1],
       [ 6,  5, 17,  3,  4, 11],
       [ 9,  1, 15,  3,  3,  3],
       [ 8,  2,  9,  4,  5,  3],
       [ 3,  6, 19, 10,  1,  1],
       [ 6,  5, 11,  3,  4, 11],
       [ 6,  1, 10,  3,  3,  3],
       [16,  2,  9,  4,  2,  3],
       [-6,  6, 19,  4,  1,  1],
       [ 6,  5, 16,  3,  6, 11],
       [ 6

https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6 

In [26]:
num_generations = 10

num_parents_mating = 4

fitness = np.empty(new_population.shape[0])

best_outputs = []

for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
        
    for i in range(len(new_population)):
        fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])
        
    #print("Fitness")    
    #print(fitness)   
    
    max_index_fitness = np.where(fitness == np.max(fitness))
    best_pop = new_population[max_index_fitness]
    best_out = np.max(np.sum(best_pop*XX1, axis=1))
    
    best_outputs.append(best_out)

    # The best result in the current iteration.
    #print("Best result : ", np.max(np.sum(new_population*XX1, axis=1)))
    
    # Selecting the best parents in the population for mating.
    
    parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
        
    
    prob = np.random.randint(low = 0, high = 100)
    
    if prob < 90 :
        
        print("Parents")    
        print(parents)  
 
     # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], weights_size))
    
    #print("offspring_crossover")    
    #print(offspring_crossover)
 
     # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)
# Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.


for i in range(len(new_population)):
        fitness[i] = cal_pop_fitness(YY1, XX1, new_population[i])
                
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])

import matplotlib.pyplot
matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()

Parents
[[ 8.  2.  9.  4.  4.  3.]
 [ 8.  2.  9.  5.  2.  3.]
 [ 6.  1. 15.  7.  3.  3.]
 [ 3.  6. 19.  4.  2.  3.]]
Parents
[[ 8.  2. 15.  5.  3.  3.]
 [ 8.  2.  9.  4.  4.  3.]
 [ 8.  2.  9.  5.  4.  3.]
 [ 8.  2.  9.  5.  4.  3.]]
Parents
[[ 8.  2.  9.  5.  3.  3.]
 [ 8.  2. 12.  5.  3.  3.]
 [ 8.  2.  6.  5.  3.  3.]
 [ 8.  2. -1.  5.  4.  3.]]
Parents
[[ 6.  2. 12.  5.  3.  3.]
 [ 8.  2.  9.  5.  3.  3.]
 [ 8.  2.  9.  5.  3.  3.]
 [ 8.  2.  9.  5.  3.  3.]]
Parents
[[ 6.  2. 12.  5.  3.  3.]
 [ 6.  2. 13.  5.  3.  3.]
 [ 7.  2. 12.  5.  3.  3.]
 [ 8.  2. 10.  5.  3.  3.]]


ValueError: Unable to coerce to DataFrame, shape must be (50, 6): given (2, 6)

In [ ]:
print("Best solution : ", new_population[best_match_idx, :])

In [ ]:
pop_size

In [ ]:
parents.shape[0]

In [ ]:
fitness

In [ ]:
for instance in range(4):
    print(instance)

In [ ]:
parents.shape[0]

In [ ]:
# Mutations on the offsprings:

mutation_probability = 3 # In percent

for i in range(40):
    
    prob = np.random.randint(low = 0, high = 100)
    if prob < mutation_probability :
        print(i)
        


In [ ]:
np.max(np.sum(new_population[1]**XX1, axis = 1))

In [ ]:
best_outputs.append(np.max(np.sum(best_pop*XX1, axis=1)))